## Flatten a dataframe into a single row

https://stackoverflow.com/questions/50556537/flatten-dataframe-into-a-single-row - actually, no - this is the Pandas solution.

Try this instead -

https://sparkbyexamples.com/spark/how-to-pivot-table-and-unpivot-a-spark-dataframe/

In [2]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [3]:
sqlContext

In [4]:
import numpy as np # linear algebra
import os
import math
import copy
import random

import time

from pyspark.ml.feature import StringIndexer, StandardScaler,VectorAssembler
from pyspark.ml import Pipeline

from pyspark.sql import functions as F
import pyspark.sql.functions as f

from pyspark.sql.functions import udf, col
from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector

from pyspark.sql.functions import rand
from pyspark.sql.types import ArrayType, FloatType,IntegerType, DataType, DoubleType, MapType

from pyspark.mllib.evaluation import MulticlassMetrics

from astropy.io import fits
from astropy.table import Table
from astropy.visualization import astropy_mpl_style
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord

maindir is /home/hduser/.astropy/config
maindir is /home/hduser/.astropy/config
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache


In [5]:
iType=IntegerType()
dType=DoubleType()
fType=FloatType()

aType=ArrayType(fType)

In [6]:
from time import time, clock
class Timer:
    """
    a simple class for printing time (s) since last call
    """
    def __init__(self):
        self.t0=time()
        self.t1=clock()
        self.elapsed = 1
        self.elapsedCPU = 1
    
    def start(self):
        self.t0=time()
        self.t1=clock()
        
    def stop(self):
        t1=time()
        t2=clock()
        print("Elapsed {:2.1f}s, CPU {:2.1f}s".format(t1-self.t0, t2-self.t1))
        self.elapsed = t1-self.t0
        self.elapsedCPU = t2-self.t1

timer=Timer()

In [7]:
def GetDetailArrays(fitsFilename, raType, decType, spectraType):
    sqlStmt=("""
        select sda_detail_array, sda_detail_type
        from Sparkfits_detail_arrays
        where sda_filename='{}'
    """).format(fitsFilename)

    detailsDF=sqlContext.sql(sqlStmt)
    
    raArray=np.array(detailsDF.filter(f.col("sda_detail_type") == raType)\
                     .select(f.col("sda_detail_array")).collect() )
    raArray=raArray.reshape(raArray.shape[2])

    decArray=np.array(detailsDF.filter(f.col("sda_detail_type") == decType)\
                      .select(f.col("sda_detail_array")).collect() )
    decArray=decArray.reshape(decArray.shape[2])

    spectraArray=np.array(detailsDF.filter(f.col("sda_detail_type") == spectraType)\
                          .select(f.col("sda_detail_array")).collect() )
    spectraArray=spectraArray.reshape(spectraArray.shape[2])

    return raArray, decArray, spectraArray

In [8]:
def GetSubCube2(fitsFilename, decType,spectraType, raArray, decArray, spectraArray, CubeSize):
    timer.start()
    ## Extract our parameters
    hiRa=CubeSize[0]
    loRa=CubeSize[1]
    hiDec=CubeSize[2]
    loDec=CubeSize[3]
    loFreq=CubeSize[4]
    hiFreq=CubeSize[5]
    
    def CreateNewHeader():
        return True
    
    global raSelectRange
    raSelectRange=np.where(np.logical_and(raArray >= loRa, raArray <= hiRa ))
    raSelectList=raSelectRange[0].tolist()
    
    raHeaderIndex=raArray[raSelectList[0]]
    naxis1=len(raSelectList)
    
    #
    # Now we need to create the array or actual RA values to pass into SoFiA
    #
    
    raDF=sqlContext.sql("""
    with data as 
    (
        select posexplode(sda_detail_array)
        from sparkfits_detail_arrays
        where sda_filename='image.restored.i.SB2338.V2.cube.fits'
        and sda_detail_type='{}'
    )
    select pos as sda_index, float(col) as sda_ra from data

    """.format(raType)      )
    
    raDF.filter((raDF.sda_ra <= hiRa ) & (raDF.sda_ra >= loRa )).registerTempTable("RA")
    
    #
    # This query pivots the independent RA values we want into a single list element we
    # can include as a Spark Column
    #
    
    filteredRaDF=sqlContext.sql("""
        with rawData as
        (
            select 1 grp,
            map(

                'sda_ra', sda_ra
            ) as kv
            from RA
        )
        select 
            grp,
            collect_list(float(a.kv['sda_ra'])) as ra
        from rawData a
        group by grp
    """)
    
    decDF=sqlContext.sql("""
    with data as 
    (
        select posexplode(sda_detail_array)
        from sparkfits_detail_arrays
        where sda_filename='image.restored.i.SB2338.V2.cube.fits'
        and sda_detail_type='{}'
    )
    select pos as sda_index, float(col) as sda_declination from data

    """.format(decType)      )
    
    freqDF=sqlContext.sql("""
    with data as 
    (
        select posexplode(sda_detail_array)
        from sparkfits_detail_arrays
        where sda_filename='image.restored.i.SB2338.V2.cube.fits'
        and sda_detail_type='{}'
    )
    select pos as sda_index, float(col) as sda_Frequency_hz from data

    """.format(spectraType)      )
    

    decFilterDF   = decDF.filter((decDF.sda_declination <= hiDec ) & (decDF.sda_declination >= loDec ))
    freqFilterDF  = freqDF.filter((freqDF.sda_Frequency_hz <= hiFreq ) \
                              & (freqDF.sda_Frequency_hz >= loFreq ))

    # decFilterDF.registerTempTable("DECLINATIONS")
    PositionDF=decFilterDF.crossJoin(filteredRaDF)
    PositionDF.registerTempTable("POSITIONS")
    freqFilterDF.registerTempTable("FREQUENCIES")

    
    decHeaderIndex=decFilterDF.groupby().max('sda_declination').collect()[0].asDict()['max(sda_declination)']
    freqHeaderIndex=freqFilterDF.groupby().min('sda_Frequency_hz').collect()[0].asDict()['min(sda_Frequency_hz)']
    naxis2=decFilterDF.count()
    naxis4=freqFilterDF.count()
    
    myImageDF=sqlContext.sql("select * from sparkfits_images where spi_filename='{}' ".format(fitsFilename))
    myImageDF.registerTempTable("IMAGES")

    subCubeSQLDF=sqlContext.sql("""
    select /*+ BROADCAST(POSITIONS), BROADCAST(FREQUENCIES) */
    spi_index, POSITIONS.ra, POSITIONS.sda_declination, IMAGES.spi_image, 
    IMAGES.spi_filename, IMAGES.spi_band, FREQUENCIES.sda_Frequency_hz
    from IMAGES
        INNER JOIN FREQUENCIES
            on IMAGES.spi_band == FREQUENCIES.sda_index
        INNER JOIN POSITIONS
            on IMAGES.spi_index == POSITIONS.sda_index

        ORDER BY spi_band, spi_index
    """ )#.explain() .format(hiFreq, loFreq, hiDec, loDec)
    
    # We still have to apply the Ra selectin criteria
    
    subCubeSQLDF=subCubeSQLDF\
    .withColumn("raSelectRange", f.array(  [f.col("spi_image")[i]  for i in raSelectList  ] ))\
    .select("spi_index","ra", "sda_declination","spi_image","raSelectRange","spi_filename","spi_band","sda_Frequency_hz")
    
    print("Image dataframe created")
    
    timer.stop()
    
    return subCubeSQLDF,raHeaderIndex,decHeaderIndex,freqHeaderIndex, naxis1,naxis2, naxis4

In [9]:
start_elapsed=time()
start_cpu=clock()

In [10]:
sqlContext.sql('use fits_investigation')

DataFrame[]

In [11]:
sqlContext.sql('desc sparkfits_images').show()

+--------------------+------------+-------+
|            col_name|   data_type|comment|
+--------------------+------------+-------+
|           spi_index|         int|   null|
|           spi_image|array<float>|   null|
|        spi_filename|      string|   null|
|            spi_band|         int|   null|
|# Partition Infor...|            |       |
|          # col_name|   data_type|comment|
|        spi_filename|      string|   null|
|            spi_band|         int|   null|
+--------------------+------------+-------+



In [12]:
#fitsFilename='residual.i.SB8170.cube.fits.NaNs.Removed'
fitsFilename='image.restored.i.SB2338.V2.cube.fits'
raType='RA---SIN'
decType='DEC--SIN'
spectraType='Hz'

In [13]:
raArray, decArray, spectraArray = GetDetailArrays(fitsFilename, raType, decType, spectraType)

In [14]:
print("Range of Ra {} to {}".format( max(raArray), min(raArray)))
print("Range of Declination {} to {}".format( max(decArray), min(decArray)))
print("Frequency ranges in {} - {} {}".format(spectraType, min(spectraArray), max(spectraArray))) 

Range of Ra 337.9596862792969 to 328.57586669921875
Range of Declination -42.05752944946289 to -48.32979202270508
Frequency ranges in Hz - 1376499968.0 1424481536.0


In [15]:
hiDec=-46.7
loDec=-47.2
hiRa=332.4
loRa=331.4
loFreq=spectraArray[7]
hiFreq=spectraArray[12]

CubeSize=[hiRa,loRa,hiDec,loDec,loFreq,hiFreq]

In [16]:
subCubeDF,raHeaderIndex,decHeaderIndex,freqHeaderIndex,naxis1, naxis2, naxis4 \
=GetSubCube2(fitsFilename, decType,spectraType, raArray, decArray, spectraArray, CubeSize)

Image dataframe created
Elapsed 3.3s, CPU 0.3s


In [17]:
if False:
    subCubeDF.show()

In [18]:
subCubeDF.registerTempTable("collatedImages")

In [19]:
if False:
    timer.start()
    subCubeDF.write.mode('overwrite').format('parquet').saveAsTable('collatedImages')
    timer.stop()

### NOTE!

In order to maintain order when we construct the collect_lists, we need to use DISTRIBUTE BY to guarantee that all rowss for a particular frequency are routed to the same reducer; otherwise we can't guarantee that the order will be maintained.

See https://stackoverflow.com/questions/45092576/how-to-use-order-by-with-collect-set-operation-in-hive

#### REMEMBER!

It's not a relational database where the ordering in usually implicit!

### NOTE!

Drop the temporary tables when you've finished with them to free up resources. 

For temporary views spark.catalog.dropTempView("df") - these are the session scoped tables, ie. the ones we use in this notebook.

For global views spark.catalog.dropGlobalTempView("df")

We can also free up resources for dataframes we have finished using, i.e. df.unpersist()

In [20]:
compress1=sqlContext.sql("""
with rawData as
(
    select sda_Frequency_hz, spi_index, ra as rightAscension,
    map(
        
        'dec', sda_declination
    ) as kva,
    map(
        'pixels', raSelectRange
    ) as kvi
    from collatedImages
    distribute by sda_Frequency_hz
    sort by sda_Frequency_hz, spi_index
)
select 
    sda_Frequency_hz as frequency,rightAscension,
    -- ntile(4) over (sort by sda_Frequency_hz) as bins,
    collect_list(float(a.kva['dec']))as declination
    ,collect_list(array(a.kvi['pixels']))as pixs
from rawData a
group by frequency, rightAscension
""")

if False:
    compress1.show()

compress1.registerTempTable("compress_one")

In [21]:
if False:
    sqlContext.sql("""
    select * from compress_one
    """).show()

### Now, let's take it that one step further and compress those three frequencies into one row

In [22]:
compress2=sqlContext.sql("""
select
    --bins,
    rightAscension, declination,
    collect_list(array(float(a.kvi['frequencies']))) as frequencies,
    collect_list(array(a.kva['pixs'])) as pixels
from (
    select
        --bins,
        rightAscension, declination,
        map('frequencies', frequency) as kvi,
        map('pixs', pixs) as kva
    from compress_one
    distribute by rightAscension
    sort by frequency
) a
group by --bins,
rightAscension, declination
""")

In [23]:
compress2.printSchema()

root
 |-- rightAscension: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- declination: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- frequencies: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)
 |-- pixels: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |    |-- element: float (containsNull = true)



In [24]:
if False:
    compress2.show()

In [25]:
if False:
    timer.start()
    compress2.write.mode('overwrite').format('parquet').saveAsTable('compression_madness')
    timer.stop()

## Function to create a FITS file from the data

This is the base unction that will be wrapped in a UDF to create a FITS file object on every row of the dataframe; when trhe UDF gets created, the input parameter inputDF will need to be changed to allow for the element from the dataframe.

In [26]:
def CreateFITSSubCube(inputDF, fitsFilename):
    #
    # Functions we need
    #
    def RetrieveHeader(hduData):
        # Create header file

        hdu=fits.PrimaryHDU()
        hduHeader=hdu.header

        def isFloat(string):
            try:
                float(string)
                return True
            except ValueError:
                return False
        for row in hduData.rdd.toLocalIterator():

            if row.sfh_value=='True':
                v=True
            elif isFloat(row.sfh_value):
                v=float(row.sfh_value)
                pass
            elif row.sfh_value.isnumeric():
                v=int(row.sfh_value)
                pass
            else:
                v=row.sfh_value
            #print(row.key, v) #row.value, row.value.isnumeric(), isFloat(row.value), bool(row.value))
            hduHeader[row.sfh_key] = (v, row.sfh_comment)
            pass

        return hduHeader
    
    def GetBaselineHeader(fitsFilename):
        hduData=sqlContext.sql("""
            select * 
            from sparkfits_fits_headers
            where sfh_fits_file='{}' 
            order by sfh_index
        """.format(fitsFilename)
        )
        
        retrievedHeader=RetrieveHeader(hduData)
        return retrievedHeader
    
    def UpdateNewHeader(newheader, retrievedHeader, raIdx, decIdx, freqIdx):
        def isFloat(string):
            try:
                float(string)
                return True
            except ValueError:
                return False

        for i in np.arange(len(retrievedHeader)):
            try:
                newheader[list(retrievedHeader.keys())[i]]
                # print("Exists!", list(jheader.keys())[i], jheader[int(i)],  jheader.comments[int(i)])
            except Exception as e:
                # print(list(header.keys())[i], header[int(i)],  header.comments[int(i)])
                jkey = list(retrievedHeader.keys())[i]

                if isFloat(retrievedHeader[int(i)]):
                    jval=float(retrievedHeader[int(i)])
                    pass
                elif retrievedHeader[int(i)].isnumeric():
                    jval=int(row.value)
                    pass
                else:
                    jval = retrievedHeader[int(i)]


                jcom = retrievedHeader.comments[int(i)]

                newheader[jkey] = (jval, jcom)
                
        wcs=WCS(retrievedHeader)
        crpix1,crpix2,crpix3,crpix4=wcs.wcs_world2pix(raIdx,decIdx ,1, freqIdx,1, ra_dec_order=True)

        newheader['CRPIX1']=float(crpix1)
        newheader['CRPIX2']=float(crpix2)
        newheader['CRPIX3']=float(1.0)
        newheader['CRPIX4']=float(crpix4)

        return newheader
        
    
    #
    # Collect the arrays from the dataframe
    #
    timer.start()
    print("Commencing data extraction from dataframe...")
    
    ra=np.array(inputDF.select('rightAscension').collect(), dtype='float64')
    declination=np.array(inputDF.select('declination').collect(), dtype='float64') #.reshape(r,sequence_len)
    frequencies=np.array(inputDF.select('frequencies').collect(), dtype='float64') #.reshape(r,sequence_len)
    pixels=np.array(inputDF.select('pixels').collect(), dtype='float64')
    
    #
    # Reshape the pixels array to the 3 dimensional numpy array
    #
    print("Pixels old shape")
    print(pixels.shape)
    pixels=pixels.reshape(pixels.shape[2], pixels.shape[3], pixels.shape[4], pixels.shape[6])
    print("Pixels new shape")
    print(pixels.shape)
    print("Data extracted from dataframe")
    timer.stop()
    
    
    #
    # Create the corner pixel indexes for the new header
    #
    timer.start()
    raHeaderIndex=ra[0][0][0]
    decHeaderIndex=declination[0][0][0]
    freqHeaderIndex=frequencies[0][0][0][0]
    
    print("New position indexes created")
    timer.stop()
    #
    # Create the new FITS object
    #
    timer.start()
    newhdu=fits.PrimaryHDU(data=pixels)
    newheader=newhdu.header
    
    print("new base fits object created")
    timer.stop()
    #
    # Get the baseline header
    #
    timer.start()
    baselineHeader=GetBaselineHeader(fitsFilename)
    print("baseline header extracted")
    timer.stop()
    
    #
    # Create the new header
    #
    
    timer.start()
    newHeader=UpdateNewHeader(newheader, baselineHeader, raHeaderIndex, decHeaderIndex, freqHeaderIndex)
    print("new header created")
    timer.stop()
    
    #
    # Return the new FITS object
    #
    
    return newhdu

In [26]:
newFITS=CreateFITSSubCube(compress2, fitsFilename)

Commencing data extraction from dataframe...
Pixels old shape
(1, 1, 6, 1, 451, 1, 597)
Pixels new shape
(6, 1, 451, 597)
Data extracted from dataframe
Elapsed 274.4s, CPU 0.2s
New position indexes created
Elapsed 0.0s, CPU 0.0s
new base fits object created
Elapsed 0.0s, CPU 0.0s
baseline header extracted
Elapsed 4.4s, CPU 0.0s
new header created
Elapsed 0.1s, CPU 0.1s


indices in parameterized keywords must not have leading zeroes. [astropy.wcs.wcs]
indices in parameterized keywords must not have leading zeroes. [astropy.wcs.wcs]
indices in parameterized keywords must not have leading zeroes. [astropy.wcs.wcs]
indices in parameterized keywords must not have leading zeroes. [astropy.wcs.wcs]
indices in parameterized keywords must not have leading zeroes. [astropy.wcs.wcs]
indices in parameterized keywords must not have leading zeroes. [astropy.wcs.wcs]
indices in parameterized keywords must not have leading zeroes. [astropy.wcs.wcs]
indices in parameterized keywords must not have leading zeroes. [astropy.wcs.wcs]
indices in parameterized keywords must not have leading zeroes. [astropy.wcs.wcs]
indices in parameterized keywords must not have leading zeroes. [astropy.wcs.wcs]
indices in parameterized keywords must not have leading zeroes. [astropy.wcs.wcs]
indices in parameterized keywords must not have leading zeroes. [astropy.wcs.wcs]
indices in param

In [27]:
newFITS.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    4 / number of array dimensions                     
NAXIS1  =                  597                                                  
NAXIS2  =                  451                                                  
NAXIS3  =                    1                                                  
NAXIS4  =                    6                                                  
EXTEND  =                    T                                                  
BSCALE  =                  1.0 / PHYSICAL = PIXEL*BSCALE + BZERO                
BZERO   =                  0.0                                                  
BUNIT   = 'Jy/beam '           / Brightness (pixel) unit                        
EQUINOX =               2000.0                                                  
RADESYS = 'FK5     '        

In [27]:
end_elapsed=time()
end_cpu=clock()

print(end_elapsed - start_elapsed)

41.713807344436646


In [ ]:
compress2.show()

## Function to create the new header as a dictionary object

So we can pass this in to the UDF

In [77]:
def createHeaderDictionary(fitsFilename):
    hduData=sqlContext.sql("""
        select sfh_key, sfh_value 
        from sparkfits_fits_headers
        where sfh_fits_file='{}' 
        order by sfh_index
    """.format(fitsFilename)
    )

    headerDict=hduData.rdd.collectAsMap()
    
    return hduData , headerDict

In [79]:
#xDict , 

hduData, xDict = createHeaderDictionary(fitsFilename)

In [81]:
hduData.registerTempTable("headers")

In [82]:
sqlContext.sql("""
select * from headers
""").show()

+-------+---------------+
|sfh_key|      sfh_value|
+-------+---------------+
| SIMPLE|           True|
| BITPIX|            -32|
|  NAXIS|              4|
| NAXIS1|           5607|
| NAXIS2|           5654|
| NAXIS3|              1|
| NAXIS4|           2592|
| BSCALE|            1.0|
|  BZERO|            0.0|
|  BUNIT|        Jy/beam|
|EQUINOX|         2000.0|
|RADESYS|            FK5|
|LONPOLE|          180.0|
|LATPOLE|-44.83073888889|
|PC01_01|            1.0|
|PC02_01|            0.0|
|PC03_01|            0.0|
|PC04_01|            0.0|
|PC01_02|            0.0|
|PC02_02|            1.0|
+-------+---------------+
only showing top 20 rows



In [84]:
sqlContext.sql("""
    select 1 as grp, sfh_index, sfh_key, sfh_value 
    from sparkfits_fits_headers
    where sfh_fits_file='{}' 
    order by sfh_index
""".format(fitsFilename)
).registerTempTable("headers")

In [85]:
sqlContext.sql("""
--desc sparkfits_fits_headers
select * from headers
""").show()

+---+---------+-------+---------------+
|grp|sfh_index|sfh_key|      sfh_value|
+---+---------+-------+---------------+
|  1|        0| SIMPLE|           True|
|  1|        1| BITPIX|            -32|
|  1|        2|  NAXIS|              4|
|  1|        3| NAXIS1|           5607|
|  1|        4| NAXIS2|           5654|
|  1|        5| NAXIS3|              1|
|  1|        6| NAXIS4|           2592|
|  1|        7| BSCALE|            1.0|
|  1|        8|  BZERO|            0.0|
|  1|       10|  BUNIT|        Jy/beam|
|  1|       11|EQUINOX|         2000.0|
|  1|       12|RADESYS|            FK5|
|  1|       13|LONPOLE|          180.0|
|  1|       14|LATPOLE|-44.83073888889|
|  1|       15|PC01_01|            1.0|
|  1|       16|PC02_01|            0.0|
|  1|       17|PC03_01|            0.0|
|  1|       18|PC04_01|            0.0|
|  1|       19|PC01_02|            0.0|
|  1|       20|PC02_02|            1.0|
+---+---------+-------+---------------+
only showing top 20 rows



In [92]:
originalHeader=sqlContext.sql("""
with rawData as
(
    select grp, sfh_index, 
    map(
        
        'key', sfh_key
    ) as kva,
    map(
        'value', sfh_value
    ) as kvi
    from headers
    distribute by grp
    sort by grp, sfh_index
)
select 
    grp,
    collect_list(string(a.kva['key']))as keys
    ,collect_list(string(a.kvi['value']))as values
from rawData a
group by grp
""").select("keys","values")

originalHeader.show()

+--------------------+--------------------+
|                keys|              values|
+--------------------+--------------------+
|[SIMPLE, BITPIX, ...|[True, -32, 4, 56...|
+--------------------+--------------------+



In [94]:
compress3 = compress2.crossJoin(originalHeader)

In [95]:
compress3.printSchema()

root
 |-- rightAscension: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- declination: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- frequencies: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: float (containsNull = true)
 |-- pixels: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |    |-- element: float (containsNull = true)
 |-- keys: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- values: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [104]:
def CreateFITSSubCubeUDF(ra, decl, freq, pixels, keys, values, fitsFilename):    
    #
    # Functions we need
    #
    #
    # Collect the arrays from the dataframe
    #
    timer.start()
    print("Commencing data extraction from dataframe...")
    
    ra=np.array(ra, dtype='float64')
    declination=np.array(decl, dtype='float64') #.reshape(r,sequence_len)
    frequencies=np.array(freq, dtype='float64') #.reshape(r,sequence_len)
    pixels=np.array(pixels, dtype='float64')
    keys=np.array(keys, dtype='str')
    values=np.array(values, dtype='str')
    
    #
    # Reshape the pixels array to the 3 dimensional numpy array
    #
    pixels=pixels.reshape(pixels.shape[0], pixels.shape[1], pixels.shape[2], pixels.shape[4])
    print("Data extracted from dataframe")
    timer.stop()
    
    
    #
    # Create the corner pixel indexes for the new header
    #
    timer.start()
    raHeaderIndex=ra[0]
    decHeaderIndex=declination[0]
    freqHeaderIndex=frequencies[0]
    
    print("New position indexes created")
    timer.stop()
    #
    # Create the new FITS object
    #
    timer.start()
    newhdu=fits.PrimaryHDU(data=pixels)
    #newheader=newhdu.header
    
    print("new base fits object created")
    timer.stop()
    #
    # Get the baseline header
    #
    timer.start()
    #baselineHeader=GetBaselineHeader(fitsFilename)
    print("baseline header extracted")
    timer.stop()
    
    #
    # Create the new header
    #
    
    timer.start()
    #newHeader=UpdateNewHeader(newheader, baselineHeader, raHeaderIndex, decHeaderIndex, freqHeaderIndex)
    print("new header created")
    timer.stop()
    
    #
    # Return the new FITS object
    #
    
    return len(keys) #len(ra) #newhdu
    
    
    
    
    
    

In [105]:
from pyspark.sql.types import MapType
test = udf(CreateFITSSubCubeUDF, ArrayType(IntegerType() ) )

In [106]:
dfout=compress3.withColumn('calc_mod', test(compress3.rightAscension, \
                                           compress3.declination, \
                                           compress3.frequencies, \
                                           compress3.pixels, \
                                           compress3.keys, \
                                           compress3.values, \
                                           f.lit(fitsFilename))).select('calc_mod')

In [107]:
dfout.show()

+--------+
|calc_mod|
+--------+
|    null|
+--------+



In [35]:
d={}

In [36]:
d

{}

In [ ]:
newFITS.writeto('hdfs:///user/hduser/sofiaTest.fits')

In [5]:
print(tm.time()-start_elapsed)

38.26019620895386


In [33]:
frequencies.shape

(1, 1, 3, 1)

In [80]:
pixels.shape


(1, 1, 3, 1, 451, 1, 597)

In [82]:
pixels.shape

(3, 1, 451, 597)

In [37]:
from astropy.io import fits
from astropy.table import Table
from astropy.visualization import astropy_mpl_style
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord

maindir is /home/hduser/.astropy/config
maindir is /home/hduser/.astropy/config
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache


In [84]:
newheader

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    4 / number of array dimensions                     
NAXIS1  =                  597                                                  
NAXIS2  =                  451                                                  
NAXIS3  =                    1                                                  
NAXIS4  =                    3                                                  
EXTEND  =                    T                                                  

In [45]:
len(declination[0][0])

451

In [85]:
timer.start()

sqlStmt=("""
        select * 
        from sparkfits_fits_headers
        where sfh_fits_file='{}' 
        order by sfh_index
    """.format(fitsFilename)
        )

hduData=sqlContext.sql(sqlStmt)
def RetrieveHeader(hduData):
    # Create header file

    hdu=fits.PrimaryHDU()
    hduHeader=hdu.header

    def isFloat(string):
        try:
            float(string)
            return True
        except ValueError:
            return False
    for row in hduData.rdd.toLocalIterator():

        if row.sfh_value=='True':
            v=True
        elif isFloat(row.sfh_value):
            v=float(row.sfh_value)
            pass
        elif row.sfh_value.isnumeric():
            v=int(row.sfh_value)
            pass
        else:
            v=row.sfh_value
        #print(row.key, v) #row.value, row.value.isnumeric(), isFloat(row.value), bool(row.value))
        hduHeader[row.sfh_key] = (v, row.sfh_comment)
        pass
    
    return hduHeader

retrievedHeader = RetrieveHeader(hduData)
retrievedHeader

timer.stop()

Elapsed 6.1s, CPU 0.0s


In [86]:
def isFloat(string):
    try:
        float(string)
        return True
    except ValueError:
        return False
    
for i in np.arange(len(retrievedHeader)):
    try:
        newheader[list(retrievedHeader.keys())[i]]
        # print("Exists!", list(jheader.keys())[i], jheader[int(i)],  jheader.comments[int(i)])
    except Exception as e:
        # print(list(header.keys())[i], header[int(i)],  header.comments[int(i)])
        jkey = list(retrievedHeader.keys())[i]
        
        if isFloat(retrievedHeader[int(i)]):
            jval=float(retrievedHeader[int(i)])
            pass
        elif retrievedHeader[int(i)].isnumeric():
            jval=int(row.value)
            pass
        else:
            jval = retrievedHeader[int(i)]
            
        
        jcom = retrievedHeader.comments[int(i)]
        
        newheader[jkey] = (jval, jcom)

In [87]:
newheader

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    4 / number of array dimensions                     
NAXIS1  =                  597                                                  
NAXIS2  =                  451                                                  
NAXIS3  =                    1                                                  
NAXIS4  =                    3                                                  
EXTEND  =                    T                                                  
BSCALE  =                  1.0 / PHYSICAL = PIXEL*BSCALE + BZERO                
BZERO   =                  0.0                                                  
BUNIT   = 'Jy/beam '           / Brightness (pixel) unit                        
EQUINOX =               2000.0                                                  
RADESYS = 'FK5     '        

In [58]:
raHeaderIndex=ra[0][0][0]
decHeaderIndex=declination[0][0][0]
freqHeaderIndex=frequencies[0][0][0][0]

In [67]:
len(ra[0][0]), len(declination[0][0]), len(frequencies[0][0]), crpix1, crpix2, crpix3

(597, 451, 3, array(3328.01916777), array(1569.38620247), array(0.))

In [72]:
float(crpix1), float(crpix2), float(crpix3), float(crpix4)

(3328.0191677683442, 1569.3862024655527, 0.0, -74330.00208473358)

In [88]:
wcs=WCS(retrievedHeader)
crpix1,crpix2,crpix3,crpix4=wcs.wcs_world2pix(raHeaderIndex,decHeaderIndex ,1, freqHeaderIndex,1, ra_dec_order=True)

newheader['CRPIX1']=float(crpix1)
newheader['CRPIX2']=float(crpix2)
newheader['CRPIX3']=float(1.0)
newheader['CRPIX4']=float(crpix4)

newWCS=WCS(newheader)

In [89]:
newheader

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    4 / number of array dimensions                     
NAXIS1  =                  597                                                  
NAXIS2  =                  451                                                  
NAXIS3  =                    1                                                  
NAXIS4  =                    3                                                  
EXTEND  =                    T                                                  
BSCALE  =                  1.0 / PHYSICAL = PIXEL*BSCALE + BZERO                
BZERO   =                  0.0                                                  
BUNIT   = 'Jy/beam '           / Brightness (pixel) unit                        
EQUINOX =               2000.0                                                  
RADESYS = 'FK5     '        

In [90]:
newWCS

WCS Keywords

Number of WCS axes: 4
CTYPE : 'RA---SIN'  'DEC--SIN'  'STOKES'  'FREQ'  
CRVAL : 332.6475708333  -44.83073888889  1.0  1376500000.056  
CRPIX : 3328.019167768344  1569.386202465553  1.0  -74330.00208473358  
PC1_1 PC1_2 PC1_3 PC1_4  : 1.0  0.0  0.0  0.0  
PC2_1 PC2_2 PC2_3 PC2_4  : 0.0  1.0  0.0  0.0  
PC3_1 PC3_2 PC3_3 PC3_4  : 0.0  0.0  1.0  0.0  
PC4_1 PC4_2 PC4_3 PC4_4  : 0.0  0.0  0.0  1.0  
CDELT : -0.001111111111111  0.001111111111111  1.0  18518.51799989  
NAXIS : 597  451  1  3

In [94]:
newhdu.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    4 / number of array dimensions                     
NAXIS1  =                  597                                                  
NAXIS2  =                  451                                                  
NAXIS3  =                    1                                                  
NAXIS4  =                    3                                                  
EXTEND  =                    T                                                  
BSCALE  =                  1.0 / PHYSICAL = PIXEL*BSCALE + BZERO                
BZERO   =                  0.0                                                  
BUNIT   = 'Jy/beam '           / Brightness (pixel) unit                        
EQUINOX =               2000.0                                                  
RADESYS = 'FK5     '        

In [96]:
newhdu